## 1 connect to database

In [19]:
import pymysql
conn = pymysql.connect(host='127.0.0.1',port=3306,user='root',password='123456',database='CertificateDB',charset='utf8')
conn

## 2 query data

In [20]:
import pandas as pd
pd.read_sql("show tables",con=conn)

,Tables_in_CertificateDB
0,certificate


In [21]:
pd.read_sql("desc certificate",con=conn)

,Field,Type,Null,Key,Default,Extra
0,id,int(11),NO,PRI,None,auto_increment
1,host,varchar(256),NO,,None,
2,open443,varchar(256),YES,,None,
3,error,varchar(256),YES,,None,
4,ssl_error,varchar(256),YES,,None,
5,certificate_version,varchar(10),YES,,None,
6,certificate_algorithm,varchar(256),YES,,None,
7,issuer_country,varchar(256),YES,,None,
8,issued_organization,varchar(256),YES,,None,
9,public_key_type,varchar(256),YES,,None,


In [22]:
def exec_sql(conn, sql):
    with conn.cursor() as cursor:
        cursor.execute(sql)
    conn.commit()

    
def query_sql(conn,sql):
    with conn.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchall()
    return result

## 3 Analysis the data

### 3.1 check whether host443 open

In [30]:
pd.read_sql("""SELECT open443, COUNT(open443) AS num
    FROM certificate
    GROUP BY open443""",con=conn)

,open443,num
0,0,6519
1,1,107337
2,null,5770


## 3.2 analysis error

In [34]:
pd.read_sql("""SELECT error,COUNT(error) AS num
    FROM certificate
    WHERE NOT (error = 'null')
    GROUP BY error
    ORDER BY num DESC""",con=conn)

,error,num
0,[Errno -5] No address associated with hostname,5039
1,"[('SSL routines', 'tls_process_server_certific...",3168
2,Failed: Misconfiguration SSL/TLS,668
3,[Errno -3] Temporary failure in name resolution,486
4,[Errno -2] Name or service not known,245
5,"[('SSL routines', 'ssl3_read_bytes', 'sslv3 al...",165
6,"[('SSL routines', 'ssl3_read_bytes', 'tlsv1 al...",132
7,"[('SSL routines', 'ssl3_get_record', 'wrong ve...",125
8,[Errno 111] Connection refused,85
9,"[('SSL routines', 'tls_process_server_certific...",50


In [33]:
pd.read_sql("""SELECT ssl_error,COUNT(ssl_error) AS num
    FROM certificate 
    WHERE NOT (ssl_error = 'null' or ssl_error = '0')
    GROUP BY ssl_error
    ORDER BY num DESC""",con=conn)

,ssl_error,num
0,20,1409
1,18,1201
2,10,1173
3,19,50


## 3.3 analysis issuers(country and organization)

In [26]:
pd.read_sql("""SELECT issuer_country,
       COUNT(issuer_country) AS num,
       COUNT(issuer_country)*100.0/(SELECT COUNT(*) 
                                    FROM certificate
                                    WHERE NOT (issued_organization = 'null')) AS percentage
FROM certificate
WHERE NOT (issuer_country = 'null')
GROUP BY issuer_country
ORDER BY num DESC""",con=conn)

,issuer_country,num,percentage
0,US,85174,82.77518
1,GB,8653,8.40930
2,BE,3866,3.75712
3,CN,1494,1.45192
4,PL,789,0.76678
5,AT,610,0.59282
6,NL,458,0.44510
7,JP,419,0.40720
8,LV,309,0.30030
9,FR,302,0.29349


In [27]:
pd.read_sql("""SELECT issued_organization,
       COUNT(issued_organization) AS num,
       COUNT(issued_organization)*100.0/(SELECT COUNT(*) 
                                         FROM certificate
                                         WHERE NOT (issued_organization = 'null')) AS percentage
    FROM certificate
    WHERE NOT (issued_organization = 'null')
    GROUP BY issued_organization
    ORDER BY num DESC""",con=conn)

,issued_organization,num,percentage
0,Let's Encrypt,30613,29.75082
1,"Cloudflare, Inc.",26347,25.60497
2,DigiCert Inc,11974,11.63677
3,Sectigo Limited,8502,8.26255
4,Amazon,6565,6.38010
...,...,...,...
119,CERTDATA SERVICOS DE INFORMACAO LTDA,1,0.00097
120,Isimtescil Bilisim Anonim Sirketi,1,0.00097
121,Abitab S.A.,1,0.00097
122,Dreamcommerce S.A.,1,0.00097


## 3.4 analysis algorithm and public key & bits

In [28]:
pd.read_sql("""SELECT certificate_algorithm,
       COUNT(certificate_algorithm) AS num,
       COUNT(certificate_algorithm)*100.0/(SELECT COUNT(*) 
                                           FROM certificate 
                                           WHERE NOT (certificate_algorithm = 'null')) AS percentage
    FROM certificate
    WHERE NOT (certificate_algorithm = 'null')
    GROUP BY certificate_algorithm
    ORDER BY num DESC""",con=conn)

,certificate_algorithm,num,percentage
0,sha256WithRSAEncryption,71737,69.71661
1,ecdsa-with-SHA256,26441,25.69632
2,ecdsa-with-SHA384,3503,3.40434
3,sha384WithRSAEncryption,1169,1.13608
4,sha512WithRSAEncryption,48,0.04665


In [ ]:
pd.read_sql("""SELECT public_key_bits,
       public_key_type,
       COUNT(public_key_bits) AS num,
       COUNT(public_key_bits)*100.0/(SELECT COUNT(*) 
                                     FROM certificate 
                                     WHERE NOT (public_key_bits = 'null')) AS percentage
FROM certificate
WHERE NOT (public_key_bits = 'null')
GROUP BY public_key_bits
ORDER BY num DESC""",con=conn)

## 3.5 check expire status

In [35]:
pd.read_sql("""SELECT expired,COUNT(expired) AS num
    FROM certificate
    GROUP BY expired
    ORDER BY num DESC""",con=conn)

,expired,num
0,0,102898
1,null,16728


In [36]:
pd.read_sql("""SELECT validity_days, COUNT(validity_days) AS num
    FROM certificate
    WHERE NOT (validity_days = 'null')
    GROUP BY validity_days
    ORDER BY num DESC""",con=conn)

,validity_days,num
0,89,31423
1,364,24422
2,365,11864
3,396,5928
4,394,5615
...,...,...
440,149,1
441,118,1
442,32,1
443,75,1


## 3.6 CRL check

In [37]:
pd.read_sql("""SELECT crl_status, COUNT(crl_status) AS num
    FROM certificate
    WHERE NOT (crl_status = 'null')
    GROUP BY crl_status
    ORDER BY num DESC""",con=conn)

,crl_status,num
0,GOOD,60620
1,FAILED,42269
2,REVOKED,9


In [38]:
pd.read_sql("""SELECT crl_reason, COUNT(crl_reason) AS num
    FROM certificate
    WHERE NOT (crl_reason = 'null')
    GROUP BY crl_reason
    ORDER BY num DESC""",con=conn)

,crl_reason,num
0,CRL ERROR: Not Found CRL Extension,42189
1,CRL ERROR: No connection adapters were found f...,25
2,CRL ERROR: No connection adapters were found f...,15
3,CRL ERROR: No connection adapters were found f...,13
4,CRL ERROR: No connection adapters were found f...,8
5,CRL ERROR: No connection adapters were found f...,7
6,FAILED,7
7,CRL ERROR: No connection adapters were found f...,4
8,"CRL ERROR: Invalid URL '<Name(C=JP,O=SECOM Tru...",2
9,CRL ERROR: No connection adapters were found f...,2


## 3.7 OCSP check

In [39]:
pd.read_sql("""SELECT ocsp_status, COUNT(ocsp_status) AS num
    FROM certificate
    WHERE NOT (ocsp_status = 'null')
    GROUP BY ocsp_status
    ORDER BY num DESC""",con=conn)

,ocsp_status,num
0,GOOD,102207
1,REVOKED,29


In [40]:
pd.read_sql("""SELECT ocsp_error, COUNT(ocsp_error) AS num
    FROM certificate
    WHERE NOT (ocsp_error = 'null')
    GROUP BY ocsp_error
    ORDER BY num DESC""",con=conn)

,ocsp_error,num
0,Error: get_ocsp_response: Request timeout for ...,433
1,Error: get_ocsp_response: Unknown Connection E...,11
2,Error: timed out,8
3,Error: get_ocsp_response: Request timeout for ...,3
4,Error: get_ocsp_response: Unknown Connection E...,3
...,...,...
203,Error: get_certificate_chain: boke112.com did ...,1
204,Error: get_certificate_chain: Client Certifica...,1
205,Error: get_certificate_chain: f-droid.org did ...,1
206,Error: get_certificate_chain: Connection to co...,1
